In [3]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2023-07-06 14:58:49

This model breaks up the "right of way" parcel(s) in a given parcel dataset
into smaller segments. 

Inputs are parcels (with right of way value/field identified), roads, and road
vertices that are pre-processed in ArcGIS Pro.

Created by Lydia Eldrige, modified by RBowers 2023.07.06

"""
import arcpy
import os
from sys import argv


Set parameters - change out below

In [12]:
#set parameters
roads_fp = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\RightOfWay_Segmentation.gdb\\EOTROADS_MAPC" #roads

#road vertices for this project were generated manually in ArcGIS Pro using the  ‘EOTROADS_MAPC’ layer 
# by using ‘Feature Vertices to Point’ (point type: start and end vertices). 
road_vertices_fp = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\RightOfWay_Segmentation.gdb\\Rd_Vertices_MAPC" #road vertices

#set geography name for file output names
geography_name = 'Bolton' #what is the geography? this outputs in file names

#parcels data
parcel_fp = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb\\Bolton_Parcels_MassGIS_LU"

#within parcel data, what is the field that identifies whether a parcel is a ROW?
row_field = 'POLY_TYPE' 


#SQL expression that identifies the field and value signifying a right of way parcel. 
# change out field names, right of way values, and add or subtract values as needed
#row_expression = "'poly_type' = 'ROW' Or 'poly_type'  = 'PRIV_ROW'"
row_expression = "POLY_TYPE = 'ROW' Or POLY_TYPE  = 'PRIV_ROW'"

outWorkspace =  "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\RightOfWay_Segmentation.gdb"  #set where intermediate data should be exported
finalWorkspace = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\ROW_model_output.gdb" #set where final data should be exported



ROW Segmentation model - brought in from ArcGIS ModelBuilder with additional comments and clarity

In [15]:
def rowsegmentationmodel(Geography, 
                         Roads_Data, 
                         Road_Vertices, 
                         Parcel_Data, 
                         Expression):  # row_segmentation_model

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    
    # Process: Select Layer By Attribute(Select Layer By Attribute) (management) - follows row_expression to select parcels signified as ROW
    print('selecting row layers...')
    _1_Parcels_Clip_Layer, Count_2_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Parcel_Data, 
                                                                              where_clause=Expression)
    
    # Process: Spatial Join (Spatial Join) (analysis)
    # The clipped roads are spatially joined to the clipped intersections (8) using “Merge Rule: Minimum” on the output field “CLASS” 
    # (this will allow the intersections to be buffered according to their functional class) 
    print('spatial join...')
    _Geography_Intersec_Rd_SpJoin = os.path.join(outWorkspace, Geography + '_Intersec_Rd_SpJoin')



    arcpy.analysis.SpatialJoin(target_features=Road_Vertices, 
                               join_features=Roads_Data, 
                               out_feature_class=_Geography_Intersec_Rd_SpJoin, 
                               join_type="KEEP_COMMON")
    
    
    # Process: Calculate Field (Calculate Field) (management)
    #A new field (“buffdist”) is created in the output of the spatial join (9) (figure 4). The values are calculated based on the lowest road class3 at each intersection and determined as follows: 
    ### 185 meters for class 1 (Limited Access Highway) and 2 (Multi-lane Highway, not limited access) 
    ### 50 meters for class 3 (Other numbered route) and 4 (Major road - arterials and collectors)  
    ### 25 meters for class 5 and 6 (Minor street or road). 

    print('calculating field...')
    _Value_Intersec_Rd_SpJoin_3_ = arcpy.management.CalculateField(in_table=_Geography_Intersec_Rd_SpJoin, 
                                                                   field="buffdist", 
                                                                   expression="Reclass(!CLASS!)", 
                                                                   code_block="""def Reclass(CLASS):
                                                                                if (CLASS > 0 and CLASS <= 2):
                                                                                    return 185
                                                                                elif (CLASS > 2 and CLASS <= 4):
                                                                                    return 50
                                                                                else:
                                                                                    return 25
                                                                                """, 
                                                                    field_type="SHORT")[0]
    

    # Process: Pairwise Buffer (Pairwise Buffer) (analysis)
    # The resulting “buffdist” values are used to buffer the intersections (10) (figure 5) after which the buffers are dissolved (below). 
    print('buffering...')
    _Geography_Intersect_Buffer = os.path.join(outWorkspace, Geography + '_Intersect_Buffer')



    arcpy.analysis.PairwiseBuffer(in_features=_Value_Intersec_Rd_SpJoin_3_, 
                                  out_feature_class=_Geography_Intersect_Buffer, 
                                  buffer_distance_or_field="buffdist")

    
    # Process: Pairwise Dissolve (Pairwise Dissolve) (analysis)
    print('pairwise dissolve...')
    _Geography_Intersec_Buff_Dissolve = os.path.join(outWorkspace, Geography + '_Intersec_Buff_Dissolve')

    arcpy.analysis.PairwiseDissolve(in_features=_Geography_Intersect_Buffer, 
                                    out_feature_class=_Geography_Intersec_Buff_Dissolve)

    # Process: Export Features (Export Features) (conversion)
    print('exporting features...')
    _Geography_ROWparcels = os.path.join(outWorkspace, Geography + '_ROWparcels')


    arcpy.conversion.ExportFeatures(in_features=_1_Parcels_Clip_Layer, 
                                    out_features=_Geography_ROWparcels, 
                                    field_mapping="")
   
    # Process: Pairwise Clip (4) (Pairwise Clip) (analysis)
    #The resulting polygon layer is clipped to the ROW parcels . 
    print('pairwise clip...')
    _Geography_InterBuff_Clip = os.path.join(outWorkspace, Geography + '_InterBuff_Clip')

 
    arcpy.analysis.PairwiseClip(in_features=_Geography_Intersec_Buff_Dissolve, 
                                clip_features=_Geography_ROWparcels, 
                                out_feature_class=_Geography_InterBuff_Clip)
    
    # Process: Union (Union) (analysis)
    # Lastly, the clipped buffers and ROW parcels are unioned (13). 
    print('union...')
    _Geography_parcel_buffer_union = os.path.join(outWorkspace, Geography + '_parcel_buffer_union')

    if arcpy.Exists(_Geography_parcel_buffer_union):
        arcpy.Delete_management(_Geography_parcel_buffer_union)

    arcpy.analysis.Union(in_features=[[_Geography_InterBuff_Clip, ""], 
                                      [_Geography_ROWparcels, ""]], 
                                      out_feature_class=_Geography_parcel_buffer_union, 
                                      gaps="GAPS")
    
    
    # Process: Multipart To Singlepart (Multipart To Singlepart) (management)
    # The resulting layer is converting into single part polygons
    print('multipart to single part...')
    _Geography_union_result = os.path.join(outWorkspace, Geography + '_union_result')


    arcpy.management.MultipartToSinglepart(in_features=_Geography_parcel_buffer_union, 
                                           out_feature_class=_Geography_union_result)
    
    

    # Process: Select Layer By Attribute (3) (Select Layer By Attribute) (management)
    # polygons are then selected by attribute for a shape area > 500 meters
    print('select layer by attribute...')
    _1_union_result_Layer, Count_3_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=_Geography_union_result, 
                                                                              where_clause="Shape_area < 500")

    
    # Process: Eliminate (Eliminate) (management)
    # Using ‘Eliminate’, all selected polygons are combined with the polygon they share the longest border with 
    # The result is the final product (segmented right of way parcels) (d), named in convention “ROW_segmentation_TownName.” 
    print('eliminate and export final layer...')
    ROW_segmentation_Geography_ = os.path.join(finalWorkspace, ("ROW_segmentation_" + Geography))
    print(ROW_segmentation_Geography_)

    arcpy.management.Eliminate(in_features=_1_union_result_Layer, 
                               out_feature_class=ROW_segmentation_Geography_)


In [16]:
with arcpy.EnvManager(scratchWorkspace=outWorkspace, workspace=outWorkspace):
                        rowsegmentationmodel(Geography= geography_name, 
                                            Roads_Data = roads_fp,  
                                            Road_Vertices = road_vertices_fp, 
                                            Parcel_Data = parcel_fp,
                                            Expression = row_expression)



selecting row layers...
spatial join...
calculating field...
buffering...
pairwise dissolve...
exporting features...
pairwise clip...
union...
multipart to single part...
select layer by attribute...
eliminate and export final layer...
K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\ROW_model_output.gdb\ROW_segmentation_Bolton
